refer to this for installation https://github.com/abetlen/llama-cpp-python#installation-with-openblas--cublas--clblast--metal 

In [1]:
%pip install llama_index
%pip install llama-cpp-python
%pip install transformers
%pip install torch
%pip install tqdm
%pip install pypdf


  Using cached aiostream-0.5.2-py3-none-any.whl.metadata (9.9 kB)
  Using cached dataclasses_json-0.5.14-py3-none-any.whl.metadata (22 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached numpy-1.26.1-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pandas-2.1.2-cp310-cp310-win_amd64.whl.metadata (18 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached tiktoken-0.5.1-cp310-cp310-win_amd64.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.8.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached urllib3-1.26.18-py2.py3-none-any.whl.metadata (48 kB)
  Using cached marshmallow-3.20.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached wrapt-1.15.0-cp310-cp310-win_amd64.whl (36 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_

In [2]:
import logging
import sys

from llama_index.callbacks import CallbackManager, LlamaDebugHandler
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

logging.basicConfig(stream=sys.stdout, level=logging.INFO)  # Change INFO to DEBUG if you want more extensive logging
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

llm = LlamaCPP(
    model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q5_K_M.gguf",
    
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    
    temperature=0.0,
    max_new_tokens=1024,
    
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,  # note, this sets n_ctx in the model_kwargs below, so you don't need to pass it there.
    
    # kwargs to pass to __call__()
    generate_kwargs={},
    
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1}, # I need to play with this and see if it actually helps
    
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [3]:
# Create an index of your documents
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

storage_directory = "./storage"

documents = SimpleDirectoryReader('./testdata').load_data()

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)


index = VectorStoreIndex.from_documents(documents, service_context=service_context)
# Persist the index to disk
index.storage_context.persist(persist_dir=storage_directory)

c:\Users\User\Downloads\GitHub\s34-cms-portal\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**********
Trace: index_construction
    |_node_parsing ->  0.142614 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000995 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.000998 seconds
      |_chunking ->  0.000996 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000995 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.001 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000994 seconds
      |_chunking ->  0.001001 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.000995 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.001002 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000997 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.0 seconds
      |_chunking ->  0.000993 seconds
      |_chunking ->  0.001004 seconds
      |_chunking ->  0.000995 seconds
      |_chunking

In [32]:
# Now you can load the index from disk when needed, and not rebuild it each time.
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index import StorageContext, load_index_from_storage

# transcript_directory = "transcripts/ancient-aliens-official"
storage_directory = "./storage"

service_context = ServiceContext.from_defaults(llm=llm, chunk_size=1024,
                                               embed_model="local",
                                               callback_manager=callback_manager)

storage_context = StorageContext.from_defaults(persist_dir=storage_directory)
index = load_index_from_storage(storage_context, service_context=service_context)

INFO:llama_index.indices.loading:Loading all indices.
Loading all indices.
**********
Trace: index_construction
**********


In [34]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

query = "Generate 5 multiple choice questions for the activists learning style with answers, regarding PDPA. \n" \
        "Explain why these 5 questions is suitable for activists learner."

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  274.981212 seconds
      |_retrieve ->  0.034909 seconds
        |_embedding ->  0.02892 seconds
      |_synthesize ->  274.946303 seconds
        |_templating ->  0.0 seconds
        |_llm ->  274.937819 seconds
**********


<b>  Sure, I'd be happy to help! Based on the context information provided, here are five multiple choice questions that would be suitable for Activists learners regarding PDPA (Personal Data Protection Act):

1. What is the main purpose of the Personal Data Protection Act (PDPA)?
a) To protect individuals' personal data from unauthorized use or disclosure
b) To provide a framework for the collection, storage, and use of personal data
c) To regulate the flow of personal data across borders
d) To establish a national identity card system

Answer: a) To protect individuals' personal data from unauthorized use or disclosure

Explanation: Activists learners tend to prefer hands-on, practical experiences and enjoy taking risks. This question is suitable for them because it focuses on the main purpose of PDPA, which is to protect individuals' personal data, and it provides a clear and concise answer that can be applied in real-world situations.

2. What is considered as sensitive personal data under PDPA?
a) Name, address, and contact information only
b) All information related to an individual's health, financial status, and personal preferences
c) Only information that is publicly available
d) Any information that is deemed sensitive by the organization

Answer: b) All information related to an individual's health, financial status, and personal preferences

Explanation: Activists learners tend to be curious and enjoy exploring new ideas. This question is suitable for them because it provides a comprehensive list of sensitive personal data under PDPA, which can help them understand the scope of the Act and its implications on their work.

3. What is the difference between "data controller" and "data processor" under PDPA?
a) Data controller is responsible for collecting personal data, while data processor is responsible for storing and processing it
b) Data controller is responsible for storing and processing personal data, while data processor is responsible for collecting it
c) Data controller is the individual who provides personal data, while data processor is the organization that collects it
d) Data controller and data processor are interchangeable terms

Answer: a) Data controller is responsible for collecting personal data, while data processor is responsible for storing and processing it

Explanation: Activists learners tend to enjoy taking on leadership roles and being in the limelight. This question is suitable for them because it provides a clear distinction between the roles of data controller and data processor under PDPA, which can help them understand their responsibilities and accountability under the Act.

4. What is the penalty for non-compliance with PDPA?
a) Warning letter only
b) Fine of up to $10,000
c) Imprisonment for a maximum of 5 years
d) All of the above

Answer: d) All of the above

Explanation: Activists learners tend to be risk-takers and enjoy taking challenges. This question is suitable for them because it provides a range of penalties for non-compliance with PDPA, which can help them understand the gravity of the Act and the potential consequences of non-compliance.

5. How does PDPA protect individuals' personal data from unauthorized use or disclosure?
a) By requiring organizations to obtain explicit consent from individuals before collecting their personal data
b) By providing a secure online portal for individuals to access and manage their personal data
c) By establishing a national identity card system that links all personal data to an individual's identity
d) By allowing individuals to opt-out of the collection and use of their personal data

Answer: a) By requiring organizations to obtain explicit consent from individuals before collecting their personal data

Explanation: Activists learners tend to be action-oriented and enjoy exploring new ideas. This question is suitable for them because it provides a clear explanation of how PDPA protects individuals' personal data, which can help them understand the practical implications of the Act and its benefits to society.</b>

In [35]:
print(response)

  Sure, I'd be happy to help! Based on the context information provided, here are five multiple choice questions that would be suitable for Activists learners regarding PDPA (Personal Data Protection Act):

1. What is the main purpose of the Personal Data Protection Act (PDPA)?
a) To protect individuals' personal data from unauthorized use or disclosure
b) To provide a framework for the collection, storage, and use of personal data
c) To regulate the flow of personal data across borders
d) To establish a national identity card system

Answer: a) To protect individuals' personal data from unauthorized use or disclosure

Explanation: Activists learners tend to prefer hands-on, practical experiences and enjoy taking risks. This question is suitable for them because it focuses on the main purpose of PDPA, which is to protect individuals' personal data, and it provides a clear and concise answer that can be applied in real-world situations.

2. What is considered as sensitive personal data 

In [37]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

r_query = "Generate 5 multiple choice questions for the reflector learning style with answers, regarding PDPA. \n" \
        "Explain why these 5 questions is suitable for reflector learner."

r_response = query_engine.query(r_query)
display(Markdown(f"<b>{r_response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  242.617619 seconds
      |_retrieve ->  0.041919 seconds
        |_embedding ->  0.036941 seconds
      |_synthesize ->  242.574699 seconds
        |_templating ->  0.0 seconds
        |_llm ->  242.565753 seconds
**********


<b>  Certainly! Based on the context information provided, here are five multiple choice questions that would be suitable for a reflector learning style, specifically related to PDPA (Personal Data Protection Act):

1. Which of the following is the primary goal of the Personal Data Protection Act?
a) To protect the privacy of individuals' personal data
b) To promote the use of technology in businesses
c) To increase the efficiency of data collection and storage
d) To reduce the risk of data breaches

Answer: a) To protect the privacy of individuals' personal data

Explanation: Reflector learners prefer to think deeply about information before acting, so they would benefit from questions that allow them to consider the purpose and goals of PDPA. This question allows them to reflect on the primary objective of the Act and its impact on personal data privacy.

2. What is the key principle of the Personal Data Protection Act?
a) Consent must be obtained before collecting personal data
b) Personal data must be collected only for specified purposes
c) Personal data must be accurate and up-to-date
d) Personal data must be stored securely and protected from unauthorized access

Answer: b) Personal data must be collected only for specified purposes

Explanation: Reflector learners prefer to understand the underlying principles and concepts before applying them, so this question allows them to reflect on the key principle of PDPA and its significance in protecting personal data.

3. How should organizations obtain consent from individuals before collecting their personal data?
a) By providing a clear and concise explanation of the purpose and scope of the data collection
b) By offering a checkbox or opt-in option on a website or form
c) By requiring individuals to sign a written agreement
d) By asking for explicit consent through a phone call or email

Answer: a) By providing a clear and concise explanation of the purpose and scope of the data collection

Explanation: Reflector learners prefer to have all relevant information before making a decision, so this question allows them to consider the best approach for obtaining informed consent from individuals.

4. What is the appropriate way to handle sensitive personal data?
a) To store it securely and protect it from unauthorized access
b) To share it with third-party vendors for analysis and marketing purposes
c) To delete it immediately after it is no longer needed
d) To use it for any purpose deemed appropriate by the organization

Answer: a) To store it securely and protect it from unauthorized access

Explanation: Reflector learners prefer to think carefully before acting, so this question allows them to reflect on the appropriate handling of sensitive personal data and the potential risks and consequences of mishandling such data.

5. What is the role of the Personal Data Protection Officer (PDPO) in ensuring PDPA compliance?
a) To review and approve all data collection and storage practices
b) To provide training and guidance to employees on PDPA requirements
c) To conduct regular audits to ensure compliance with PDPA
d) To serve as the main point of contact for individuals with questions or concerns about their personal data

Answer: b) To provide training and guidance to employees on PDPA requirements

Explanation: Reflector learners prefer to understand the roles and responsibilities of key individuals in an organization, so this question allows them to reflect on the role of the PDPO in ensuring PDPA compliance and the importance of proper training and guidance for employees.</b>

In [38]:
print(r_response)

  Certainly! Based on the context information provided, here are five multiple choice questions that would be suitable for a reflector learning style, specifically related to PDPA (Personal Data Protection Act):

1. Which of the following is the primary goal of the Personal Data Protection Act?
a) To protect the privacy of individuals' personal data
b) To promote the use of technology in businesses
c) To increase the efficiency of data collection and storage
d) To reduce the risk of data breaches

Answer: a) To protect the privacy of individuals' personal data

Explanation: Reflector learners prefer to think deeply about information before acting, so they would benefit from questions that allow them to consider the purpose and goals of PDPA. This question allows them to reflect on the primary objective of the Act and its impact on personal data privacy.

2. What is the key principle of the Personal Data Protection Act?
a) Consent must be obtained before collecting personal data
b) Pers

In [40]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

t_query = "Generate 5 multiple choice questions for the theorist learning style with answers, regarding PDPA. \n" \
        "Explain why these 5 questions is suitable for theorist learner."

t_response = query_engine.query(t_query)
display(Markdown(f"<b>{t_response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  289.325651 seconds
      |_retrieve ->  0.062833 seconds
        |_embedding ->  0.056849 seconds
      |_synthesize ->  289.260823 seconds
        |_templating ->  0.0 seconds
        |_llm ->  289.252844 seconds
**********


<b>  Certainly! Based on the context information provided, here are five multiple choice questions that would be suitable for a Theorist learning style, specifically related to PDPA (Personal Data Protection Act):

1. Which of the following is the primary goal of the Personal Data Protection Act (PDPA)?
a) To protect individuals' personal data from unauthorized use or disclosure
b) To provide a framework for organizations to collect and use personal data without restrictions
c) To promote the development of new technologies that can enhance data privacy and security
d) To impose strict penalties on organizations that violate personal data protection regulations

Answer: a) To protect individuals' personal data from unauthorized use or disclosure

Explanation: Theorists tend to prefer a more analytical and conceptual approach to learning, so they would appreciate questions that allow them to explore the underlying principles and concepts of PDPA. This question allows them to understand the primary objective of PDPA and its significance in protecting individuals' personal data.

2. Which of the following is an essential requirement for organizations under the Personal Data Protection Act (PDPA)?
a) To obtain explicit consent from individuals before collecting their personal data
b) To provide clear and concise information about the purpose and scope of personal data collection
c) To establish a comprehensive data security framework to protect personal data from unauthorized access or disclosure
d) To regularly review and update their personal data protection policies and procedures

Answer: b) To provide clear and concise information about the purpose and scope of personal data collection

Explanation: Theorists tend to appreciate questions that allow them to explore the interrelationship amongst elements, so this question allows them to understand the essential requirements for organizations under PDPA and how they relate to each other.

3. What is the primary responsibility of data protection officers (DPOs) under the Personal Data Protection Act (PDPA)?
a) To ensure that personal data is collected and used in compliance with PDPA regulations
b) To provide training and guidance to employees on personal data protection policies and procedures
c) To conduct regular audits to ensure compliance with PDPA regulations
d) To serve as the primary point of contact for individuals who have concerns about their personal data

Answer: a) To ensure that personal data is collected and used in compliance with PDPA regulations

Explanation: Theorists tend to enjoy exploring abstract concepts and analyzing complex situations, so this question allows them to understand the primary responsibility of DPOs under PDPA and how they can contribute to ensuring compliance with regulations.

4. Which of the following is a key principle of the Personal Data Protection Act (PDPA)?
a) That personal data should be collected only for specified, explicit, and legitimate purposes
b) That personal data should be accurate and up-to-date
c) That personal data should be protected from unauthorized access or disclosure
d) That personal data should be deleted once it is no longer needed

Answer: a) That personal data should be collected only for specified, explicit, and legitimate purposes

Explanation: Theorists tend to appreciate questions that allow them to explore the underlying principles and concepts of PDPA, so this question allows them to understand one of the key principles of PDPA and how it relates to personal data protection.

5. What is the purpose of the Personal Data Protection Act (PDPA) in relation to cross-border data transfers?
a) To prohibit cross-border data transfers altogether
b) To regulate cross-border data transfers to ensure that they are conducted in compliance with PDPA regulations
c) To allow organizations to transfer personal data freely across borders without any restrictions
d) To provide a framework for the exchange of personal data between countries

Answer: b) To regulate cross-border data transfers to ensure that they are conducted in compliance with PDPA regulations

Explanation: Theorists tend to enjoy analyzing complex situations and exploring abstract concepts, so this question allows them to understand the purpose of PDPA in relation to cross-border data transfers and how it relates to other aspects of personal data protection.</b>

In [41]:
print(t_response)

  Certainly! Based on the context information provided, here are five multiple choice questions that would be suitable for a Theorist learning style, specifically related to PDPA (Personal Data Protection Act):

1. Which of the following is the primary goal of the Personal Data Protection Act (PDPA)?
a) To protect individuals' personal data from unauthorized use or disclosure
b) To provide a framework for organizations to collect and use personal data without restrictions
c) To promote the development of new technologies that can enhance data privacy and security
d) To impose strict penalties on organizations that violate personal data protection regulations

Answer: a) To protect individuals' personal data from unauthorized use or disclosure

Explanation: Theorists tend to prefer a more analytical and conceptual approach to learning, so they would appreciate questions that allow them to explore the underlying principles and concepts of PDPA. This question allows them to understand the

In [45]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

p_query = "Generate 5 multiple choice questions for the pragmatist learning style with answers, regarding PDPA. \n" \
        "Explain why these 5 questions is suitable for pragmatist learner."

p_response = query_engine.query(p_query)
display(Markdown(f"<b>{p_response}</b>"))

Llama.generate: prefix-match hit


**********
Trace: query
    |_query ->  280.106531 seconds
      |_retrieve ->  0.034906 seconds
        |_embedding ->  0.029921 seconds
      |_synthesize ->  280.071625 seconds
        |_templating ->  0.0 seconds
        |_llm ->  280.053643 seconds
**********


<b>  Sure! Based on the context information provided, here are five multiple choice questions that would be suitable for a Pragmatist learning style, specifically related to PDPA (Personal Data Protection Act):

1. What is the most important aspect of PDPA that organizations must comply with?
a) The requirement to obtain explicit consent from individuals before collecting their personal data
b) The need to implement strict data security measures to protect personal data
c) The obligation to provide clear and concise information about the purpose and scope of personal data collection
d) The requirement to appoint a Data Protection Officer (DPO) to oversee PDPA compliance

Explanation: Pragmatists prefer practical and action-oriented learning, so they would appreciate questions that focus on the most important aspects of PDPA compliance. This question highlights the key aspect of obtaining explicit consent from individuals, which is a critical requirement for organizations to comply with.

2. How should organizations ensure that they are using personal data in a way that is compliant with PDPA?
a) By conducting regular audits and risk assessments to identify potential risks and vulnerabilities
b) By implementing strict data security measures to protect personal data
c) By providing clear and concise information about the purpose and scope of personal data collection
d) By appointing a Data Protection Officer (DPO) to oversee PDPA compliance

Explanation: Pragmatists prefer practical and action-oriented learning, so they would appreciate questions that focus on specific actions organizations can take to ensure PDPA compliance. This question highlights the importance of conducting regular audits and risk assessments to identify potential risks and vulnerabilities.

3. What is the best way for organizations to obtain explicit consent from individuals before collecting their personal data?
a) By using clear and concise language in privacy notices and consent forms
b) By providing a tick-box option for individuals to indicate their consent
c) By using visual aids such as icons or images to illustrate the purpose of personal data collection
d) By obtaining consent through a separate form or document

Explanation: Pragmatists prefer practical and action-oriented learning, so they would appreciate questions that focus on specific strategies for obtaining explicit consent. This question highlights the importance of using clear and concise language in privacy notices and consent forms to ensure individuals understand how their personal data will be used.

4. How can organizations ensure that they are protecting the privacy of individuals when collecting and processing their personal data?
a) By implementing strict data security measures to protect against unauthorized access or disclosure
b) By providing clear and concise information about the purpose and scope of personal data collection
c) By obtaining explicit consent from individuals before collecting their personal data
d) By appointing a Data Protection Officer (DPO) to oversee PDPA compliance

Explanation: Pragmatists prefer practical and action-oriented learning, so they would appreciate questions that focus on specific actions organizations can take to protect the privacy of individuals. This question highlights the importance of implementing strict data security measures to protect against unauthorized access or disclosure.

5. What should organizations do if they suspect a data breach has occurred involving personal data?
a) Immediately notify affected individuals and the relevant authorities
b) Conduct an internal investigation to determine the cause of the breach before notifying anyone
c) Ignore the incident and hope it does not become public knowledge
d) Appoint a Data Protection Officer (DPO) to oversee the response to the breach

Explanation: Pragmatists prefer practical and action-oriented learning, so they would appreciate questions that focus on specific actions organizations should take in response to a data breach. This question highlights the importance of immediately notifying affected individuals and the relevant authorities if a data breach has occurred.</b>

In [46]:
print(p_response)

  Sure! Based on the context information provided, here are five multiple choice questions that would be suitable for a Pragmatist learning style, specifically related to PDPA (Personal Data Protection Act):

1. What is the most important aspect of PDPA that organizations must comply with?
a) The requirement to obtain explicit consent from individuals before collecting their personal data
b) The need to implement strict data security measures to protect personal data
c) The obligation to provide clear and concise information about the purpose and scope of personal data collection
d) The requirement to appoint a Data Protection Officer (DPO) to oversee PDPA compliance

Explanation: Pragmatists prefer practical and action-oriented learning, so they would appreciate questions that focus on the most important aspects of PDPA compliance. This question highlights the key aspect of obtaining explicit consent from individuals, which is a critical requirement for organizations to comply with.



In [49]:
# # convert the response to a str
# response = str(response)
# r_response = str(r_response)
# t_response = str(t_response)
# p_response = str(p_response)


# # create a txt file to store the questions
# f = open("questions.txt", "w")
# f.write(response)
# f.write(r_response)
# f.write(t_response)
# f.write(p_response)
# f.close()


In [ ]:
raise Exception("Stop here")

In [ ]:
# Query your index!
from IPython.display import Markdown, display
from llama_index.prompts import PromptTemplate

query_engine = index.as_query_engine(service_context=service_context,
                                     similarity_top_k=3)

# response = query_engine.query("What do you think of Facebook's LLaMa?")
# print(response)

# query = "What is the Alien nuclear agenda? \n\n" \
#         "Please summarize the information into 3 detailed paragraphs, something suitable for a blog post."

query = "I am trying to develop a game to enhance adult learning learning \n\n" \
        "Given that the game is a narrative driven adventure game, please generate 5 multiple choice question regarding sexual harrassment \n\n"

response = query_engine.query(query)
display(Markdown(f"<b>{response}</b>"))